In [1]:
# libraries and functions.

!pip3 install pydash
!pip3 install unidecode

import json
import pandas
import pathlib
import pydash
import requests
import unidecode

def parse_array(row, col, path):

  # parse an array of dictionaries.

  return [pydash.get(x, path) for x in row[col]]

def parse_single(row, col):

  # extract single dictionary values.

  return pydash.get(row[col], 'value') 

def string_clean(row, col):

  # string normalisation for matching.

  return unidecode.unidecode(str(row[col]).upper()).strip()   

def sparql_query(query):

  # send sparql request, and formulate results into a dataframe. 

  r = requests.get('https://query.wikidata.org/sparql', params = {'format': 'json', 'query': query})
  if r.status_code == 200:
    data = pydash.get(r.json(), 'results.bindings')
    data = pandas.DataFrame.from_dict(data)
    for x in data.columns:    
      data[x] = data.apply(parse_single, col=x, axis=1)
    return data

def year_adjust(row, col, incr):

  # increment year to facilitate more matches.

  return row[col]+incr

     |████████████████████████████████| 84 kB 3.1 MB/s 
     |████████████████████████████████| 235 kB 8.3 MB/s 


In [2]:
# git clone acmi api data.

!git clone https://github.com/ACMILabs/acmi-api.git

Cloning into 'acmi-api'...
remote: Enumerating objects: 112987, done.
remote: Counting objects: 100% (56487/56487), done.
remote: Compressing objects: 100% (10089/10089), done.
remote: Total 112987 (delta 46678), reused 56158 (delta 46353), pack-reused 56500
Receiving objects: 100% (112987/112987), 463.31 MiB | 20.43 MiB/s, done.
Resolving deltas: 100% (89171/89171), done.
Checking out files: 100% (47014/47014), done.


In [3]:
# dataframe of acmi film data.

json_path = pathlib.Path.cwd() / 'acmi-api' / 'app' / 'json' / 'works'
json_data = [x for x in json_path.rglob('**/*') if x.suffix == '.json']

cols = ['id', 'acmi_id', 'title', 'production_dates', 'creators_primary']
dataframe = pandas.DataFrame(columns=cols)

for n, x in enumerate(json_data):
    with open(x) as a:
        a = json.load(a)
        if pydash.get(a, 'type') == 'Film':
            dataframe.loc[len(dataframe)] = [pydash.get(a, c) for c in cols]

for a, b in [('production_dates', 'date'), ('creators_primary', 'name')]:
    dataframe[a] = dataframe.apply(parse_array, col=a, path=b, axis=1)
    dataframe = dataframe.explode(a)

dataframe = dataframe.fillna('')
for a in ['production_dates', 'creators_primary']:
    dataframe = dataframe.loc[~dataframe[a].isin([''])]

dataframe['title'] = dataframe['title'].str.split('=').str[0].str.strip()
for x in ['title', 'creators_primary']:
    dataframe[x] = dataframe.apply(string_clean, col=x, axis=1)

dataframe['production_dates'] = dataframe['production_dates'].str[-4:]
dataframe = dataframe.loc[dataframe.production_dates.isin([str(x) for x in range(1800, 2100)])]
dataframe = dataframe.copy()
dataframe['production_dates'] = dataframe['production_dates'].astype('int64')

acmi_data = pandas.DataFrame()
for x in [-1, 0, 1]:
  adjusted = dataframe.copy()
  adjusted['production_dates'] = adjusted.apply(year_adjust, col='production_dates', incr=x, axis=1)
  acmi_data = pandas.concat([acmi_data, adjusted])

print(len(acmi_data)) 
acmi_data.sample(10)

264531


,id,acmi_id,title,production_dates,creators_primary
16234,82889,325339,WRITERS & REVOLUTIONARIES,1991,NIHON HOSO KYOKAI
29438,72712,009693,SHADOW PLAY,1962,ARTHUR CANTRILL
21980,84012,305294,SMASH PALACE,1982,NEW ZEALAND FILM COMMISSION
12139,95859,502312,DERWENT RUNS DOWN TO THE SEA,1963,DON ANDERSON
7227,66796,002860,DISASTER FLY,1977,PETER JOHNSON
34118,95156,319204,SCANDAL,1949,SHOCKIKU
30169,67104,003220,LIGHT AND SHADE,1951,YOUNG AMERICA FILMS
2789,69673,322086,LIFE IN COLD LANDS: ESKIMO VILLAGE,1955,CORONET INSTRUCTIONAL FILMS
35050,95761,501685,KNOW YOUR CHILDREN,1951,AUSTRALIAN NATIONAL FILM BOARD
18600,93940,317700,M*A*S*H,1970,INGO PREMINGER


In [4]:
# request for imdb datasets.

for x in ['title.basics.tsv.gz', 'title.crew.tsv.gz', 'name.basics.tsv.gz', 'title.principals.tsv.gz']:
  source = f'https://datasets.imdbws.com/{x}'
  result = pathlib.Path.cwd() / 'imdb' / pathlib.Path(source).name
  result.parents[0].mkdir(parents=True, exist_ok=True)

  req = requests.get(source, stream=True)
  with open(result, 'wb') as f:
      for chunk in req.iter_content(chunk_size=1024):
          if chunk:
              f.write(chunk)
              f.flush()

print('all done.')

all done.


In [5]:
# dataframe of imdb film data.

imdb_data_title = pandas.read_csv(pathlib.Path.cwd() / 'imdb' / 'title.basics.tsv.gz', delimiter='\t', low_memory=False)
imdb_data_title = imdb_data_title.loc[imdb_data_title.titleType.isin(['movie', 'tvMovie'])]
imdb_data_title = pandas.concat([imdb_data_title[['tconst', 'primaryTitle', 'startYear']].rename(columns={'primaryTitle':'title'}),
                                 imdb_data_title[['tconst', 'originalTitle', 'startYear']].rename(columns={'originalTitle':'title'})])

imdb_data_crew = pandas.read_csv(pathlib.Path.cwd() / 'imdb' / 'title.crew.tsv.gz', delimiter='\t', low_memory=False)
imdb_data_crew = pandas.concat([imdb_data_crew[['tconst', 'directors']].rename(columns={'directors':'nconst'}),
                                imdb_data_crew[['tconst', 'writers']].rename(columns={'writers':'nconst'})])
imdb_data_crew = imdb_data_crew.loc[imdb_data_crew.nconst.str.contains('nm', na=False)]
imdb_data_crew = imdb_data_crew[['tconst', 'nconst']].drop_duplicates()
imdb_data_crew['nconst'] = imdb_data_crew['nconst'].str.split(',')
imdb_data_crew = imdb_data_crew.explode('nconst')
imdb_data = pandas.merge(imdb_data_title, imdb_data_crew, on='tconst', how='left')

imdb_data_name = pandas.read_csv(pathlib.Path.cwd() / 'imdb' / 'name.basics.tsv.gz', delimiter='\t', low_memory=False)
imdb_data = pandas.merge(imdb_data, imdb_data_name, on='nconst', how='left')

imdb_data = imdb_data[['title', 'startYear', 'primaryName', 'tconst']].rename(columns={'startYear':'production_dates','primaryName':'creators_primary'})
imdb_data = imdb_data[['title', 'production_dates', 'creators_primary', 'tconst']].drop_duplicates()

for x in ['title', 'creators_primary']:
  imdb_data[x] = imdb_data.apply(string_clean, col=x, axis=1)

imdb_data = imdb_data.loc[~imdb_data.creators_primary.isin(['NAN'])]
imdb_data = imdb_data.loc[~imdb_data.production_dates.isin(['\\N'])]
imdb_data['production_dates'] = imdb_data['production_dates'].astype('int64')

print(len(imdb_data)) 
imdb_data.sample(10)

1388569


,title,production_dates,creators_primary,tconst
2595104,WOLMIDO,1982,LEE JIN-WOO,tt2226647
571305,CHACALES DE LA FRONTERA,1990,ANGEL SANCHO,tt0415700
1714674,DEUTSCHLAND IM HERBST,1978,ALF BRUSTELLIN,tt0077427
679634,GENERATION '91,2019,CHRISTINA TYNKEVYCH,tt10242126
479483,L'ODYSSEE D'ALICE TREMBLAY,2002,SYLVIE LUSSIER,tt0302882
858793,SLEAZEBAG CHEATERS,2020,DAN SALAMANTE,tt13723564
2434341,SARANGBANG SEONSOOWA EOMEONI,2007,YEONG-SEONG LIM,tt1468365
916828,DOG EAT DOG,2010,CURTIS JENSEN,tt1482434
806936,BROKEN PIPE DREAMS,2007,SAM FRIEDLANDER,tt1273799
1397241,LAST PRINCESS OF ROYAL BLOOD: TSETSENHANGRU,2008,BAYANERUUL,tt6984304


In [6]:
# merge and retrieve wikidata ids via imdb.

merged_data = pandas.merge(acmi_data, imdb_data, on=['title','production_dates','creators_primary'])

tconst_list = list(merged_data.tconst)
wikidata_result = pandas.DataFrame()
for x in range(int(len(tconst_list)/100)+1):
  chunk = ' '.join(["'"+x+"'" for x in tconst_list[x*100:(x+1)*100]])
  query = """SELECT DISTINCT ?wikidata ?tconst 
    WHERE {
      VALUES ?tconst {"""+chunk+"""} 
      ?wikidata wdt:P345 ?tconst.
      }"""
  wikidata_query = sparql_query(query)
  wikidata_result = pandas.concat([wikidata_result, wikidata_query])

wikidata_result['wikidata'] = wikidata_result['wikidata'].str.split('/').str[-1]
merged_data = pandas.merge(merged_data, wikidata_result, on='tconst')

print(len(merged_data))
merged_data.sample(10)

4837


,id,acmi_id,title,production_dates,creators_primary,tconst,wikidata
4738,84572,305896,YOUNG MAN WITH A HORN,1950,MICHAEL CURTIZ,tt0043153,Q946888
1609,81406,302492,DIAL M FOR MURDER,1954,ALFRED HITCHCOCK,tt0046912,Q496255
2982,95235,319292,THE BRAIN EATERS,1958,BRUNO VESOTA,tt0051432,Q3520094
3110,79367,300312,SILKWOOD,1983,MIKE NICHOLS,tt0086312,Q1412037
3169,88584,311166,THE WOMAN IN THE WINDOW,1944,NUNNALLY JOHNSON,tt0037469,Q1498122
293,115912,X000071,LES COUSINS,1959,CLAUDE CHABROL,tt0052708,Q141557
954,87480,309426,STREET OF SHAME,1956,KENJI MIZOGUCHI,tt0048933,Q2529456
3854,93701,317282,FANTASIA 2000,1999,GAETAN BRIZZI,tt0120910,Q30937
3580,85353,306740,TEXASVILLE,1990,PETER BOGDANOVICH,tt0103069,Q1757908
497,87778,309861,RIDICULE,1996,PATRICE LECONTE,tt0117477,Q661142


In [7]:
# dedupe.

merged_data = merged_data[['id', 'acmi_id', 'wikidata']].drop_duplicates()

print(len(merged_data))
merged_data.sample(10)

3817


,id,acmi_id,wikidata
73,115904,X000063,Q583859
3667,69675,322088,Q28485809
3153,94162,317944,Q4151182
2664,83860,305125,Q647997
2184,81259,302329,Q276769
614,94821,318848,Q1196384
2838,87404,309344,Q551762
432,86078,307686,Q1141186
912,94228,318020,Q2363623
4497,83088,304277,Q569189
